# Classifier of Classical Music Midis and Markov Model generated Midis for objective evaluation

## Setup:


Importing the MM data from Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

I just copied the MIDI processing functions from midi_utils.ipynb for the time being 

In [21]:
from IPython.display import clear_output
from fractions import Fraction
from music21 import *
import json
import pickle
import numpy as np
import os
import math


def open_midi_file(midi_file_path):
    midifile = converter.parse(midi_file_path)
    return midifile

# Function to open a midi file's sheet variant in MuseScore, requires MuseScore to be installed
# PS: If at first it doesn't work, try following http://web.mit.edu/music21/doc/installing/installWindows.html#install-music21
# If it still doesn't work, try running the function a second time for the heck of it. Environment variables be crazy.
# Params: string path to a midi file
def open_midi_file_musescore(midi_file_path):
    midi_file = open_midi_file(midi_file_path)
    midi_file.show()
    
# Function to play a midi file
# Params: string path to a midi file
def play_midi_file(midi_file_path):
    midi_file = open_midi_file(midi_file_path)
    midi_file.show("midi")

# Function to transpose a midi stream to C major if the song was originally in a major key, or A minor if it was in minor
# Params: midi stream (midi file opened with open_midi_file())
def transpose_midi_key(midi_file):
    midi_key = midi_file.analyze('key')
    if (midi_key.mode == "major"):
        get_interval = interval.Interval(k.tonic, pitch.Pitch('C'))
    if (midi_key.mode == "minor"):
        get_interval = interval.Interval(k.tonic, pitch.Pitch('A'))
    transposed_midi = midi_file.transpose(get_interval)
    return transposed_midi

# Function to convert the whole dataset to tokens and save them in a single .txt file
# If you're running this for the first time, it will take a while (more than an hour) because music21 converter.parse
# is quite slow. Subsequent times are fast, however, as the parser has pickled the midis somewhere.
# Params: string path to folder containing midis, string name of the output file 
def write_midis_to_txt_as_tokens(midi_folder_path, output_file_name, transpose_bool = False):
    
    def print_progress(progress, number_of_files, number_of_tokens):
        clear_output(wait=True)
        print("{0} / {1} generated. Number of tokens in current midi: {2}".format(progress, number_of_files, number_of_tokens))
    
    path = midi_folder_path
    filenames = os.listdir(path)

    with open(output_file_name, "w") as txt:
        progress = 1
        for filename in filenames:
            midi_as_tokens = convert_midi_to_tokens(path + filename, transpose_bool)
            print_progress(progress, len(filenames), len(midi_as_tokens))
            txt.write(' '.join(midi_as_tokens) + '\n')
            progress += 1

# Function to print the list of unique token values from a list of tokens
# Params: a list of tokens, string type of token (wait, note, stop_note) 
def get_unique_tokens(tokens, token_type):
    return set([i.split(":")[1] for i in tokens if i.startswith(token_type)])

# Checks if a value is essentially 0
def epsilon(n):
    return math.isclose(n, 0, abs_tol = 0.002)

def almost_whole(n):
    return epsilon(n % 1) or epsilon((n % 1) - 1)
    
# Function to make sure that a list of tokens contains a stop_note event for every note starting event
# Works similarly to adding a note in the tokens to midi conversion function
# Params: a list of tokens
# Returns true if all notes end properly, false if not
def validate_tokens(tokens):
    
    # Keeps track of the current token
    current_token_index = 0
    
    # Keeps track of notes that aren't stopped, if any
    not_stopped_notes = list() 
    
    for token in tokens:
        token_type = token.split(":")[0]
        token_value = token.split(":")[1]
        
        if token_type == "note":
            found_corresponding_end_note = False
            note_midi_pitch = token_value
            
            for following_token in tokens[current_token_index + 1:]:
                following_token_type = following_token.split(":")[0]
                following_token_value = following_token.split(":")[1]
                    
                if following_token_type == "stop_note":
                    stopped_note_pitch = following_token_value
                    if (note_midi_pitch == stopped_note_pitch):
                        found_corresponding_end_note = True
                        break
            
            if not found_corresponding_end_note:
                not_stopped_notes.append(token)
                
    return len(not_stopped_notes) == 0, not_stopped_notes
    
# Function to turn a midi file into text tokens
# Params: string path to a midi file, bool whether to transpose all midis or not
def convert_midi_to_tokens(midi_file_path, transpose_bool = False):
    
    # Function to remap velocity to not oversaturate the possible range of velocities
    # Completely made up values
    def remap_velocity(velocity):
        if velocity <= 42:
            return 50
        if velocity <= 84:
            return 70
        return 90
    
    def remap_tempo(tempo):
        if tempo <= 40:
            return 40
        if tempo >= 140:
            return 140
        return tempo - (tempo % 10)
            
    def velocity_change_handler(current_velocity, note_velocity, tokens_to_add):
        remapped_new_velocity = remap_velocity(note_velocity)
        if (current_velocity != remapped_new_velocity):
            current_velocity = remapped_new_velocity
            tokens_to_add.append("velocity:" + str(remapped_new_velocity))
        return tokens_to_add, current_velocity
    
    def tempo_change_handler(current_tempo, new_tempo, tokens_to_add):
        remapped_new_tempo = remap_tempo(new_tempo)
        if (current_tempo != remapped_new_tempo):
            current_tempo = remapped_new_tempo
            tokens_to_add.append("tempo:" + str(remapped_new_tempo))
        return tokens_to_add, current_tempo
    
    midi_file = open_midi_file(midi_file_path)
    
    if transpose_bool:
        midi_file = transpose_midi_key(midi_file)
    
    # A list to hold tokens
    tokens = list()
    
    # A list to keep track of which note events await a corresponding note off event
    notes_to_stop = list()
    
    # Keeps track of time since start of the midi piece
    current_offset = 0
    
    # Keeps track of velocity (note volume), used to add a velocity token every time this variable value changes
    # Theoretically can range from 0 to 127, but the possible range of values will be remapped to keep the final model simple
    # Default value of 70 is chosen because the author likes it
    current_velocity = 70
    
    # Keeps track of tempo, used to add a tempo token every time this variable value changes
    current_tempo = None
    
    # Keeps track of the offset of the previously handled midi event
    previous_offset = 0.0
    offset_changed = False
    
    # Because we are mushing different streams of MIDI events (e.g. left & right hand parts) into a single stream,
    # the tempos and time signatures are duplicated. The encoding, however, only needs to see them once.
    # previous_event is used to check if we just saw a tempo/timesig token to know if we should ignore its second occurrence.
    previous_event = None
    
    # Iterate over all midi events, sorted by offset (ascending)
    # and handle which tokens will be added to list of tokens
    for midi_event in midi_file.flat.elements:
        
        current_offset = midi_event.offset
        
        # At the end of the current loop, tokens in this list will be added to the final tokens list
        tokens_to_add = list()
        
        # Check if there are notes that should have ended between the last and current offset (included) 
        if len(notes_to_stop) != 0:
            for note_to_stop, when_to_stop in [note[:] for note in notes_to_stop]:
                if when_to_stop <= current_offset:
                    time_since_prev_offset = common.opFrac(when_to_stop - previous_offset)
                    if time_since_prev_offset > 0:
                        tokens_to_add.append("wait:" + str(time_since_prev_offset))
                        previous_offset = when_to_stop
                    tokens_to_add.append("stop_note:" + note_to_stop)
                    notes_to_stop.remove([note_to_stop, when_to_stop])
        
        # If the offset has changed by >0, account for it by adding a waiting token
        if (current_offset > previous_offset and isinstance(midi_event, (note.Note, chord.Chord))):
            offset_changed = True
            offset_change = common.opFrac(current_offset - previous_offset)
            tokens_to_add.append("wait:" + str(offset_change))
        
        if isinstance(midi_event, tempo.MetronomeMark) and not isinstance(previous_event, tempo.MetronomeMark):
            tempo_value = midi_event.number
            tokens_to_add, current_tempo = tempo_change_handler(current_tempo, tempo_value, tokens_to_add)
        
        # If the current midi event is a note, add a note token along with its midi pitch number
        # And remember when the note needs to be stopped
        if isinstance(midi_event, note.Note):
            midi_pitch = str(midi_event.pitch.midi)
            note_velocity = midi_event.volume.velocity
            
            tokens_to_add, current_velocity = velocity_change_handler(current_velocity, note_velocity, tokens_to_add)
            
            token_string = "note:" + midi_pitch
            
            note_end_offset = common.opFrac(current_offset + midi_event.duration.quarterLength)
            
            tokens_to_add.append(token_string)
            notes_to_stop.append([midi_pitch, note_end_offset])
        
        # If the current midi event is a chord, do the same as before for every note in the chord
        if isinstance(midi_event, chord.Chord):
            for individual_note in midi_event:
                midi_pitch = str(individual_note.pitch.midi)
                note_velocity = midi_event.volume.velocity
                
                tokens_to_add, current_velocity = velocity_change_handler(current_velocity, note_velocity, tokens_to_add)
                
                token_string = "note:" + midi_pitch
                
                note_end_offset = common.opFrac(current_offset + midi_event.duration.quarterLength)

                tokens_to_add.append(token_string)
                notes_to_stop.append([midi_pitch, note_end_offset])
        
        tokens.extend(tokens_to_add)
        
        # I can't for the life of me remember why offset_changed is required, but it makes everything work. Do not touch, lol.
        if offset_changed:
            offset_changed = False
            previous_offset = current_offset
        previous_event = midi_event
    
    # After iterating through all midi events, it is necessary to check for note stopping events one more time,
    # since the last midi event could have been a note starting event
    tokens_to_add = list()
    if len(notes_to_stop) != 0:
        for note_to_stop, when_to_stop in [note[:] for note in notes_to_stop]:
            time_since_prev_offset = common.opFrac(when_to_stop - previous_offset)
            if time_since_prev_offset > 0:
                tokens_to_add.append("wait:" + str(time_since_prev_offset))
                previous_offset = when_to_stop
            tokens_to_add.append("stop_note:" + note_to_stop)
            notes_to_stop.remove([note_to_stop, when_to_stop])
    tokens.extend(tokens_to_add)
    
    return tokens

# Function to convert list of text tokens to a Music21 midi stream
# Params: a list of tokens
def convert_tokens_to_midi(tokens):
    
    # A midi stream that will hold midi events converted from tokens
    midi_stream = stream.Stream()
    
    # Keeps track of the current token
    current_token_index = 0
    
    # Keeps track of offset
    current_offset = 0
    
    # Keeps track of velocity
    current_velocity = 70
    
    # The converter ignores BOS and EOS tags
    tokens = [token for token in tokens if token not in ["<EOS>", "<BOS>"]]
    
    for token in tokens:
        token_type = token.split(":")[0]
        token_value = token.split(":")[1]
        
        # Time signatures are unnecessary for playback. Commented out.
        if token_type == "timesignature":
            timesignature_value = token_value
            midi_stream.append(meter.TimeSignature(timesignature_value))
        
        if token_type == "tempo":
            tempo_value = float(token_value)
            midi_stream.append(tempo.MetronomeMark(number=tempo_value))
            
        if token_type == "velocity":
            velocity_value = float(token_value)
            current_velocity = velocity_value
        
        # Converting a note-starting token to a midi event, we need to know its duration.
        # To find the duration, we look at the following tokens until we find a corresponding stop_note token.
        # While searching for the stop_note token, we add up the values of intermediate wait tokens, denoting duration.
        # We identify the corresponding stop_note by the note's midi pitch number. 
        if token_type == "note":
            note_duration = 0
            note_midi_pitch = token_value

            for following_token in tokens[current_token_index + 1:]:
                following_token_type = following_token.split(":")[0]
                following_token_value = following_token.split(":")[1]
                
                if following_token_type == "wait":
                    wait_duration = common.opFrac(Fraction(following_token_value))
                    note_duration += common.opFrac(wait_duration)
                    
                if following_token_type == "stop_note":
                    stopped_note_pitch = following_token_value
                    if (note_midi_pitch == stopped_note_pitch):
                        new_note = note.Note(int(note_midi_pitch))  
                        new_note.quarterLength = note_duration
                        new_note.volume.velocity = current_velocity
                        midi_stream.insert(current_offset, new_note)
                        break

        if token_type == "wait":
            wait_duration = common.opFrac(Fraction(token_value))
            current_offset += common.opFrac(wait_duration)

        current_token_index += 1

    return midi_stream

## Data processing MIDI -> tokens

MM files

In [ ]:
tokens_mm = []
for i in range(500):
  midi_path = "/content/gdrive/MyDrive/test/" + str(i) + ".mid"
  midi_tokens = convert_midi_to_tokens(midi_path)
  tokens_mm.append(midi_tokens)

In [ ]:
import pickle
pkl_file = open('midi_mm.txt','wb')
pickle.dump(tokens_mm,pkl_file)


In [ ]:
from google.colab import files
files.download('/content/midi_mm.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
tokens_sub = []
for i in range(50):
  midi_path = "/content/obj_samps/LSTM_objective"+ str(i) + ".mid"
  midi_tokens = convert_midi_to_tokens(midi_path)
  tokens_sub.append(midi_tokens)

#import pickle
#pkl_file = open('midi_sub.txt','wb')
#pickle.dump(tokens_sub,pkl_file)

#from google.colab import files
#files.download('/content/midi_sub.txt') 

## Loading both of the datasets and buliding our test, validation and training sets

In [1]:
import pickle
filename = 'midi_mm.txt'
with open(filename, 'rb') as f:
    midis_mm = pickle.load(f)

for piece in midis_mm:
  piece.insert(0, "<BOS>")
  piece.append("<EOS>")
  piece.insert(0,'markov') # attach the label here as first element

500 markov pieces

In [2]:
len(midis_mm)

500

In [3]:
midis_orig = []
with open("/content/training_data.txt") as file:
    for line in file: 
        elements = line.strip().split(" ")
        elements.insert(0, "<BOS>")
        elements.append("<EOS>")
        elements.insert(0,'classic') # attach the label here as first element
        midis_orig.append(elements)

295 classical pieces

In [4]:
len(midis_orig)

295

Pooling everything together:

In [5]:
everything = []
everything.extend(midis_orig)
everything.extend(midis_mm)

In [6]:
len(everything)

795

In [7]:
everything2 = []
maxlen=0
for piece in everything:
  if len(piece)-1 < 10000:
    everything2.append(piece)
  if len(piece)-1>maxlen and len(piece)-1< 10000:
    maxlen = len(piece)-1 #-label

In [8]:
maxlen

9923

In [11]:
print(len(everything2))

541


In [12]:
k=0
for e in everything2:
  if e[0]=="markov":
    k+=1
print(k)

310


In [13]:
unique_tokens = set()
for i in everything2:
  for j in i:
    if j not in {"markov","classic"}:
      unique_tokens.add(j)
unique_tokens = list(unique_tokens)

In [24]:
token_int_pairings = {token: unique_tokens.index(token)+2 for token in unique_tokens}
token_int_pairings['markov'] = 0
token_int_pairings['classic'] = 1


everything_ints = []
for piece in everything2:
  piece_list = []
  for element in piece:
    piece_list.append(token_int_pairings[element])
  everything_ints.append(piece_list)

# Save dictionary
json.dump(dict(token_int_pairings.items()), open("./dictionary.json", 'w'))

In [23]:
len(everything_ints)

541

We are going to go with 80:10:10 training testing and validation split here:

In [24]:
import random
random.seed(92040)
random.shuffle(everything_ints)
train_set = everything_ints[0:433]
#train_set = everything_ints[0:318]
#train_set = everything_ints[0:636]
train_Y = np.array([seq[0] for seq in train_set],dtype = np.uint8)
train_X = [seq[1:] for seq in train_set]
test_set = everything_ints[433:487]
#test_set = everything_ints[318:358]
#test_set = everything_ints[636:716]
test_Y = np.array([seq[0] for seq in test_set],dtype = np.uint8)
test_X = [seq[1:] for seq in test_set]
validation_set = everything_ints[487:]
#validation_set = everything_ints[358:397]
validation_Y = np.array([seq[0] for seq in validation_set],dtype = np.uint8)
validation_X = [seq[1:] for seq in validation_set]
#validation_set = everything_ints[716:783]

In [ ]:
train_Y[0:10]

In [17]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import load_model
numpy.random.seed(7)

In [ ]:
train_X = sequence.pad_sequences(train_X, maxlen=maxlen)
test_X = sequence.pad_sequences(test_X, maxlen=maxlen)
validation_X = sequence.pad_sequences(validation_X, maxlen=maxlen)

In [ ]:
train_X = np.expand_dims(train_X, axis=-1)
test_X = np.expand_dims(test_X, axis=-1)
validation_X = np.expand_dims(validation_X, axis=-1)

In [ ]:
train_X.shape

(433, 9923, 1)

In [18]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
model = Sequential()
#model.add(Dropout(0.2))
model.add(GRU(64,input_shape=(maxlen,1)))
model.add(Dropout(0.5))
#model.add(Dropout(0.5))
#model.add(GRU(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',recall_m])
model.fit(train_X, train_Y, epochs=64, batch_size=64,validation_data=(validation_X, validation_Y))
# Final evaluation of the model
scores = model.evaluate(test_X, test_Y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
print("Recall: %.2f%%" % (scores[2]*100))

Epoch 1/64
7/7 [==============================] - 40s 5s/step - loss: 0.8771 - accuracy: 0.5238 - recall_m: 0.2649 - val_loss: 0.5617 - val_accuracy: 0.6852 - val_recall_m: 0.0000e+00
Epoch 2/64
7/7 [==============================] - 35s 5s/step - loss: 0.7890 - accuracy: 0.5522 - recall_m: 0.3796 - val_loss: 0.5572 - val_accuracy: 0.8704 - val_recall_m: 0.5882
Epoch 3/64
7/7 [==============================] - 35s 5s/step - loss: 0.7580 - accuracy: 0.6235 - recall_m: 0.5900 - val_loss: 0.5003 - val_accuracy: 0.8519 - val_recall_m: 0.5294
Epoch 4/64
7/7 [==============================] - 35s 5s/step - loss: 0.7013 - accuracy: 0.6264 - recall_m: 0.5782 - val_loss: 0.4389 - val_accuracy: 0.8519 - val_recall_m: 0.5294
Epoch 5/64
7/7 [==============================] - 35s 5s/step - loss: 0.6135 - accuracy: 0.6729 - recall_m: 0.5915 - val_loss: 0.3887 - val_accuracy: 0.8889 - val_recall_m: 0.6471
Epoch 6/64
7/7 [==============================] - 35s 5s/step - loss: 0.5633 - accuracy: 0.7136 

KeyboardInterrupt: ignored

In [ ]:
# Final evaluation of the model
scores = model.evaluate(test_X, test_Y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
print("Recall: %.2f%%" % (scores[2]*100))

Accuracy: 98.15%
Recall: 96.67%


In [ ]:
model.save('/content/model')

INFO:tensorflow:Assets written to: /content/model/assets


INFO:tensorflow:Assets written to: /content/model/assets


## Generated file test

In [15]:
import tensorflow as tf

In [19]:
another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  load_options = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
  loaded = tf.keras.models.load_model('/content/mdel/',custom_objects={'recall_m':recall_m}, options=load_options)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/

In [30]:
all_ints = []
token_int_pairings["velocity:90"] = token_int_pairings["velocity:70"]
token_int_pairings["velocity:50"] = token_int_pairings["velocity:40"]
token_int_pairings["tempo:140"]  = token_int_pairings["tempo:160"]
token_int_pairings['tempo:40'] = token_int_pairings['tempo:60']
token_int_pairings['tempo:50.0'] = token_int_pairings['tempo:60']
for tr in tokens_sub:
  tr.insert(0, "<BOS>")
  tr.append("<EOS>")
  midi_ints = [token_int_pairings[element] for element in tr]
  all_ints.append(midi_ints)
midi_toks_2 = sequence.pad_sequences(all_ints, maxlen=maxlen)
midi_ints_3 = np.expand_dims(midi_toks_2, axis=-1)

In [34]:
preds = loaded.predict(midi_ints_3)

In [32]:
with open('preds.txt', 'w') as f:
    for item in preds:
        f.write("%s\n" % item[0])

In [35]:
preds[0]

array([0.8689609], dtype=float32)

In [46]:
np.sum(loaded.predict_classes(midi_ints_3)==1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


32